# Tutorial - Running GUS in Amsterdam.
A demo of a minimal flow of GUS process in Amsterdam. Data Source: https://maps.amsterdam.nl

For more advanced use cases, please see upcoming article. For more structured experiment and scenario analyis plese see model sepcification and code documentation within docs/ folder. 

In [ ]:
# Install the GUS framework.

import sys

!{sys.executable} -m pip install pyGus

In [ ]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 3
%matplotlib inline
from pkg_resources import resource_filename
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os, sys, json
import pkgutil
import pprint

pp = pprint.PrettyPrinter(indent=1)
sns.set_theme()


### 0. Importing GUS modules

In [ ]:
from pygus.gus.models import Urban
from pygus.gus.utilities import (
    get_raster_data,
    latlng_array_to_xy,
)


## 1. Initilisation
### 1.1 Importing data on site, tree populations, experiment parameters

In [ ]:
# File that holds site/city other geolocated data and parameters
fname_config_site = resource_filename("pygus", "gus/inputs/site.json")

# File that include tree species specific parameters
fname_config_allometrics = resource_filename("pygus", "gus/inputs/allometrics.json")

# File that holds experiment parameters
fname_scenario_params = resource_filename("pygus", "gus/inputs/scenario.json")

In [ ]:
f = open(fname_config_allometrics)
config_allometrics = json.loads(f.read())

In [ ]:
f = open(fname_config_site)
config_site = json.loads(f.read())
pp.pprint(config_site)

#### 1.2 Load Trees Data, Drop unnecessary columns.

In [ ]:
url = "https://treesai-gus-public.s3.eu-west-3.amazonaws.com/inputs/amsterdam_all_trees.csv"
df_pop = pd.read_csv(url)
print(
    "Based on the source, there are {} Trees in the public Amsterdam Trees Dataset.".format(
        len(df_pop)
    )
)

In [ ]:
df_pop = df_pop[["OBJECTNUMMER", "Soortnaam_WTS", "Boomhoogte", "RADIUS", "LNG", "LAT"]]
df_pop = df_pop.rename(
    {
        "OBJECTNUMMER": "id",
        "Soortnaam_WTS": "species",
        "Boomhoogte": "height",
        "RADIUS": "radius",
        "LNG": "lng",
        "LAT": "lat",
    },
    axis=1,
)

In [ ]:
df_pop.set_index("id")
df_pop["dbh"] = df_pop["radius"] * 2.54
df_pop.tail()

#### 1.3 Take avg. Height

In [ ]:
import re

df_pop = df_pop[df_pop["height"] != "Onbekend"]

df_pop.tail()

In [ ]:
def get_average_height(height_string):
    """
    Extracts the average height from a string of the format 'X tot Y m'.
    """
    matches = re.findall(r"\d+", height_string)
    return sum(map(int, matches)) / len(matches)


df_pop["height"] = df_pop["height"].apply(get_average_height)

df_pop.tail()

#### 1.4 Map LAT,LNG to x,y POS on MESA Grid.

In [ ]:
# take lat and lng columns and convert to 2 separate numpy arrays
df_pop = latlng_array_to_xy(df_pop)

In [ ]:
url = "https://treesai-gus-public.s3.eu-west-3.amazonaws.com/inputs/species_list_amsterdam.csv"
df = pd.read_csv(url)
df = df.rename(columns={"Species": "species"})
df.tail()

### 2. Data Prep

Merge the two dataframes on the 'species' column
Map specific Tree species to their belonging categories 
to be able to use their generic allometric equations.


In [ ]:
merged_df = pd.merge(df, df_pop, on="species")
merged_df.columns
df_pop = merged_df[["id", "species", "Category", "height", "dbh", "xpos", "ypos", "lat", "lng"]]
df_pop.tail()

In [ ]:
f = open(fname_scenario_params)
scenario_params = json.loads(f.read())
pp.pprint(scenario_params)

In [ ]:
# Distribute the conditions of Trees over the df.
conditions = ["fair", "good", "excellent"] * (len(df_pop) // 3) + ["good"]
df_pop["condition"] = conditions

In [ ]:
from datetime import datetime

print(datetime.now())

In [ ]:
# df_pop['species'].isna().sum() -- 368, deleting those for this case now.
df_pop.dropna(subset=["species"], inplace=True)
df_pop.tail()

### 1.2 Creating a digital twin of the site

In [ ]:
site = Urban(df_pop, fname_config_allometrics, fname_config_site, scenario_params)

In [ ]:
initial_locations = get_raster_data(site, counts=True)
initial_biomass = get_raster_data(site, var="biomass")

sns.set_style("ticks")
fig = plt.figure(figsize=(16, 12))
plt.subplot(121)
plt.imshow(initial_locations, cmap="Greens")
plt.title("Tree Locations")
plt.subplot(122)
plt.title("Biomass Concentrations")
plt.imshow(initial_biomass, cmap="Blues")
# plt.colorbar(label = 'Biomass - Kg')
plt.show()


### 2. Running the scenario


In [ ]:
for i in range(scenario_params["time_horizon_years"]):
    site.step()


## 3. Analysis

### 3.1 Individual tree level inspections



In [ ]:
# Tree level individual variables
df_out_tree = site.datacollector.get_agent_vars_dataframe()
# df_out_tree.to_csv("trees_yearly.json", index=True)
df_out_tree.tail()

In [ ]:
Specific_Tree = df_out_tree.xs(259000, level="AgentID")
Specific_Tree.tail()

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(16, 8))
plt.subplot(221)
Specific_Tree.height.plot()
plt.xlabel("Years")
plt.ylabel("m")
plt.legend()
plt.subplot(222)
Specific_Tree.dbh.plot()
plt.xlabel("Years")
plt.ylabel("cm")
plt.legend()
plt.subplot(223)
Specific_Tree.carbon.plot()
plt.xlabel("Years")
plt.ylabel("Kg")
plt.legend()
plt.subplot(224)
Specific_Tree.dieback.plot()
plt.xlabel("Years")
plt.ylabel("ratio")
plt.legend()
plt.show()

In [ ]:
# Site level aggregate variables
df_out_site = site.datacollector.get_model_vars_dataframe()
df_out_site.tail()

In [ ]:
fig = plt.figure(figsize=(16, 8))
df_out_site.Seq.plot()
df_out_site.Released.plot()
plt.xlabel("Years")
plt.ylabel("Kg")
plt.legend()
plt.show()

In [ ]:
locations = get_raster_data(site, counts=True)
biomass = get_raster_data(site, var="biomass")

sns.set_style("ticks")
fig = plt.figure(figsize=(16, 16))
plt.subplot(221)
plt.imshow(initial_locations, cmap="Greens")
plt.title("Initial Tree Canopy")

plt.subplot(222)
plt.imshow(locations, cmap="Greens")
plt.title("Current Tree Canopy")

plt.subplot(223)
plt.title("Initial biomass concentrations")
plt.imshow(initial_biomass, cmap="Blues", vmin=0, vmax=biomass.max())
plt.colorbar(label="Biomass - Kg")

plt.subplot(224)
plt.title("Current biomass concentrations")
plt.imshow(biomass, cmap="Blues")
plt.colorbar(label="Biomass - Kg")
plt.show()